In [3]:
#import necessary tools used in the analysis
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import lightgbm as lgbm
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, KFold

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score,auc

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [2]:
#import the data from your local pc
train=pd.read_csv("TRAINKOS.csv")
test=pd.read_csv("TESTKOS.csv")
sub=pd.read_csv("SampleSubmission.csv")

In [3]:
train.head(2)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.0,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.0,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.80573,1.620483,0.322436,no


In [5]:
#to check the pandas version
pd.__version__

'1.1.2'

In [4]:
#test.head(2)...........this is optional

In [5]:
#train.info()

In [4]:
#using label encoder for changing alphabets into numbers recognised by the machine 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train['default_status'] = encoder.fit_transform(train['default_status'])
train['form_field47'] = encoder.fit_transform(train['form_field47'])
test['form_field47'] = encoder.fit_transform(test['form_field47'])

In [5]:



from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train['default_status'] = encoder.fit_transform(train['default_status'])

In [6]:
train.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,0,NaN,1.129518,0.044335,0
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,0,349.805730,1.620483,0.322436,0
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,0,NaN,1.513370,0.011640,1
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,0,89.940100,0.664452,0.082729,0
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,0


In [7]:
#dropping the apploicant ID is a very great move.
train.drop(["Applicant_ID"], 1, inplace = True)
test.drop(["Applicant_ID",], 1, inplace = True)

In [8]:
train.head()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,0.392854,2.02,0.711632,0.0,0.0,0,NaN,1.129518,0.044335,0
1,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,...,0.314281,8.08,0.183584,NaN,0.0,0,349.805730,1.620483,0.322436,0
2,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,...,0.162965,18.18,0.791136,0.0,0.0,0,NaN,1.513370,0.011640,1
3,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,...,0.488884,2.02,0.685168,NaN,0.0,0,89.940100,0.664452,0.082729,0
4,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,...,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,0


In [9]:
#fill in missing values 999 or -999 can also be used.
train=train.fillna(-1)
test=test.fillna(-1)


In [10]:
train.shape

(56000, 51)

In [11]:
test.shape

(24000, 50)

In [12]:
#concatenating the train and test set
data = pd.concat([train,test],axis=0)
data = data.reset_index(drop=True)

In [13]:

train=data[data.default_status.notnull()].reset_index(drop=True)
test=data[data.default_status.isna()].reset_index(drop=True)

In [14]:
y = train.default_status
train.drop('default_status', axis=1, inplace=True)
test.drop('default_status', axis=1, inplace=True)

In [15]:
train.shape, test.shape

((56000, 50), (24000, 50))

In [16]:
X_1, X_2, y_1, y_2 = train_test_split(train, y, test_size = 0.2, random_state = 0)

In [17]:
X_3, X_4, y_3, y_4 = train_test_split(X_2, y_2, test_size = 0.2, random_state = 0)

In [18]:
X_3.shape, y_3.shape

((8960, 50), (8960,))

In [19]:
val_model = XGBClassifier(random_seed = 10, n_estimators = 1000)

In [20]:
%%capture 
#In the real sense we want to see how the model's doing but I did this to avoid long training lines on github

val_model.fit(X_3, y_3)

In [21]:
val_pred = val_model.predict_proba(X_4)[:, 1]

In [22]:

log_loss(y_4, val_pred)

0.780115962181946

In [25]:
train.shape, test.shape, y.shape

((56000, 50), (24000, 50), (56000,))

In [24]:
from sklearn.model_selection import KFold,train_test_split,LeaveOneOut,StratifiedKFold

In [26]:
#using catboost for the training

errcb1=[]
y_pred_totcb1=[]
fold=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)#25
i=1
for train_index, test_index in fold.split(train,y):
    print(str(i) + ' iter')
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m1  = CatBoostClassifier(n_estimators=1000,eval_metric='AUC',depth=10,l2_leaf_reg=9,one_hot_max_size=50,
                              
                             learning_rate=0.15,random_seed= 10, use_best_model=False)
    m1.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=500,verbose=100)
    preds=m1.predict_proba(X_test)[:,1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb1.append((roc_auc_score(y_test,preds)))
    p1 = m1.predict_proba(test)[:,1]
    y_pred_totcb1.append(p1)
    i+=1
np.mean(errcb1)

1 iter
0:	test: 0.8105615	test1: 0.8001804	best: 0.8001804 (0)	total: 435ms	remaining: 7m 14s
100:	test: 0.9161851	test1: 0.8343471	best: 0.8348452 (93)	total: 28.8s	remaining: 4m 16s
200:	test: 0.9589303	test1: 0.8333103	best: 0.8348452 (93)	total: 1m 4s	remaining: 4m 15s
300:	test: 0.9802066	test1: 0.8303678	best: 0.8348452 (93)	total: 1m 33s	remaining: 3m 37s
400:	test: 0.9901245	test1: 0.8295353	best: 0.8348452 (93)	total: 2m 4s	remaining: 3m 6s
500:	test: 0.9957845	test1: 0.8266566	best: 0.8348452 (93)	total: 2m 31s	remaining: 2m 31s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8348452342
bestIteration = 93

err:  0.825850351625973
2 iter
0:	test: 0.8132454	test1: 0.8109531	best: 0.8109531 (0)	total: 277ms	remaining: 4m 36s
100:	test: 0.9157893	test1: 0.8409279	best: 0.8418352 (76)	total: 27.4s	remaining: 4m 3s
200:	test: 0.9572452	test1: 0.8400486	best: 0.8418352 (76)	total: 54.3s	remaining: 3m 35s
300:	test: 0.9792968	test1: 0.8391050	best: 0.8418352 (76)

0.8266308075635249

In [27]:
sub.default_status = np.mean(y_pred_totcb1, axis=0)

In [28]:


sub.to_csv('catNEW101.csv', index = False)

In [31]:
#using xgboost for the training
errcb2=[]
y_pred_totcb2=[]
fold=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in fold.split(train,y):
    print(str(i) + ' iter')
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = XGBClassifier(n_estimators=1200, objective ='binary:logistic', 
                          eta = 0.99,
                          max_depth = 6, 
                          n_jobs=-1,

                          reg_lambda = 500,
                          
                          colsample_bytree = 0.8,min_child_weight=1.7817,gamma=0.046,
    
   
                              
                            random_seed= 10)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=500,verbose=100)
    preds=m2.predict_proba(X_test)[:,1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb2.append((roc_auc_score(y_test,preds)))
    p2 = m2.predict_proba(test)[:,1]
    y_pred_totcb2.append(p2)
    i+=1
np.mean(errcb2)
                    

   

11 iter
[23:35:20] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { random_seed } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-error:0.19490	validation_1-error:0.20764
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 500 rounds.
[100]	validation_0-error:0.13064	validation_1-error:0.20157
[200]	validation_0-error:0.09087	validation_1-error:0.20246
[300]	validation_0-error:0.06119	validation_1-error:0.20764
[400]	validation_0-error:0.04163	validation_1-error:0.20603
[500]	validation_0-error:0.02833	validation_1-error:0.20407
Stopping. Best iteration:
[14]	validation_0-error:0.17818	validation_1-error:0.1958

err:  0.8323352461827545
19 iter
[23:45:02] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { random_seed } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-error:0.19670	validation_1-error:0.20736
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 500 rounds.
[100]	validation_0-error:0.12777	validation_1-error:0.20236
[200]	validation_0-error:0.09016	validation_1-error:0.20004
[300]	validation_0-error:0.06133	validation_1-error:0.20254
[400]	validation_0-error:0.04149	validation_1-error:0.20414
[500]	validation_0-error:0.02799	validation_1-error:0.20343
Stopping. Best iteration:
[18]	validation_0-error:0.17698	

0.8323082794912903

In [32]:
xgb_sub = sub.copy()

In [33]:
xgb_sub.default_status = np.mean(y_pred_totcb2, axis=0)

In [34]:
xgb_sub.to_csv('2XGB101.csv', index = False)

In [ ]:
#blending of predictions

In [35]:
blend = (xgb_sub.default_status*0.4) + (sub.default_status*0.6)

In [36]:
blend_sub = sub.copy()

In [37]:
sub.default_status = blend

In [38]:
blend_sub.default_status = blend

In [39]:
blend_sub.to_csv('BLEND101.csv', index = False)

In [40]:
reblend = ((xgb_sub.default_status*0.33) + (sub.default_status*0.67))*0.95 + (sub.default_status*0.05)

In [41]:
reblend_sub = sub.copy()

In [42]:

reblend_sub.default_status = reblend

In [43]:
reblend_sub.to_csv('NEWREBLEND101233.csv', index = False)